# Markup und/in Python

## Einleitung 

Dieses Inputreferat ist Teil des Moduls Auszeichnungssprachen im Master of Science "Digital Humanities" der Universität Trier. Es ist aber fast ebenso relevant für das Modul "Programmieren 1: Textprozessieren". 

Thema ist die Nutzung und Verarbeitung von Markup-Dateien (wie HTML oder XML) mit Python. Hierfür werden mehrere relevante Libraries vorgestellt und einfache Nutzungsbeispiele gezeigt. 

Insbesondere geht es um die folgenden Aspekte: 

1. In HTML-Dateien Informationen suchen mit Regulären Ausdrücken
1. HTML-Dateien durchsuchen oder bearbeiten mit BeautifulSoup
1. In XML-Dateien mit Python Informationen suchen: XPath in lxml
1. Validieren von XML-TEI mit Python: lxml
1. XSL-Transformation auf XML anwenden mit lxml
1. Dokumentation zu einem Schema generieren: RNG nach Markdown

Dieses Inputreferat fokussiert eher auf die Szenarien und Ergebnisse, als auf die Details der konkreten Umsetzung. Vollständige und ausführbare Code-Beispiele werden aber selbstverständlich mit angegeben. 


## HTML durchsuchen mit Regulären Ausdrücken

Für diesen Anwendungsfall verwenden wir mehrere aus dem "Archive of our Own" heruntergeladene Textdateien in HTML. Das folgenden Szenario wird berücksichtigt:  Extraktion einiger Metadaten. 

In [12]:
from os.path import join
from os.path import basename
import pandas as pd
import re
import glob

htmlfiles = join("..", "data", "aao", "*.html")

def read_html(htmlfile): 
    """
    Open and read a single HTML file.
    Returns the content as a string.
    """
    with open(htmlfile, "r", encoding="utf8") as infile: 
        html = infile.read()
    #print(html[2000:3000])
    return html

def find_metadata(html): 
    """
    Search specific metadata items in the HTML string.
    Returns one dictionary with metadata for one document. 
    """
    title = re.findall("<title>(.*?)</title>", html)[0]
    fandom = re.findall("<dt>Fandom:</dt>\n.*?<dd><a href=.*?>(.*?)</a>", html)[0]
    metadata = {"title" : title, "fandom": fandom}
    return metadata

def save_metadata(metadata): 
    """
    Transform the dictionary to a DataFrame and save as TSV.
    Returns a TSV file saved to disk. 
    """
    metadata = pd.DataFrame.from_dict(metadata).T
    print(metadata)
    with open("aao-metadata.tsv", "w", encoding="utf8") as outfile: 
        metadata.to_csv(outfile, sep="\t")

        
def main(htmlfile): 
    """
    Collect metadata for a collection of HTML files.
    """
    metadata = {}
    for htmlfile in glob.glob(htmlfiles):
        idno = basename(htmlfile).split(".")[0]
        html = read_html(htmlfile)
        metadata[idno] = find_metadata(html)
    print(metadata)
    save_metadata(metadata)

main(htmlfiles)

{'AAO-14065050': {'title': 'Lightning - Jetainia - Harry Potter - J K', 'fandom': 'Harry Potter - J. K. Rowling'}, 'AAO-78433': {'title': 'A Series of Sunsets - ChokolatteJedi - Harry Potter - Rowling', 'fandom': 'Harry Potter - Rowling'}, 'AAO-850406': {'title': 'If Harry met Jela - Enleve - Harry Potter and the', 'fandom': 'Harry Potter and the Methods of Rationality'}}
                                                          title  \
AAO-14065050          Lightning - Jetainia - Harry Potter - J K   
AAO-78433     A Series of Sunsets - ChokolatteJedi - Harry P...   
AAO-850406    If Harry met Jela - Enleve - Harry Potter and the   

                                                   fandom  
AAO-14065050                 Harry Potter - J. K. Rowling  
AAO-78433                          Harry Potter - Rowling  
AAO-850406    Harry Potter and the Methods of Rationality  


## Suchen in und Bearbeiten von HTML: mit BeautifulSoup

Mit der Library `BeatifulSoup` kann man nicht nur nach Mustern suchen, sondern die Dokumentstruktur des HTML-Dokuments als solche nutzen und wesentlich systematischer Markup-Dateien durchsuchen und bearbeiten. BeautifulSoup funktioniert dabei nicht nur mit HTML, sondern auch mit XML. 

Dokumentation: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [28]:
from bs4 import BeautifulSoup as bs


def process_html(html): 
    hsoup = bs(html, 'html.parser')

    # (1) Im HTML-Baum suchen: nach einem Element, Element-Inhalt, etc.
    #print(hsoup.h1)
    #print(hsoup.title)
    #print(hsoup.title.string)
    #print(len(hsoup.find_all("p")))

    # (2) Allen Text aus dem "body" extrahieren (ohne Metadaten)
    #print(hsoup.body.get_text())
    #print(len(hsoup.body.get_text()))
    #text = ""
    #for item in hsoup.find_all(id="chapters"): 
    #    text += item.get_text()
    #print(len(text))
    
    # (3) Das HTML-Dokument verändern
    element = hsoup.title
    #element.clear()      # removes the contents of an element
    #element.extract()    # removes the whole element
    #hsoup.title.wrap(hsoup.new_tag("titleStmt")) # Ein Element mit zusätzlichem Element umgeben
    #print(hsoup)
    
def main(htmlfiles): 
    for htmlfile in glob.glob(htmlfiles): 
        html = read_html(htmlfile)
        text = process_html(html)

main(htmlfiles)

## XML validieren mit lxml

Für diesen Anwendungsfall verwenden wir die ELTeC-Sammlungen als Datensatz. Das Szenario ist die Validierung aller Dateien in einem Ordner mit einem Schema. 

Die vollständige Textsammlung ELTeC-eng ist hier verfügbar: https://github.com/COST-ELTeC/ELTeC-eng. Im Beispielskript wird auf eine lokale Kopie verwiesen, die 8 Beispieldateien aus ELTeC-eng enthält.

Die Schemadatei ist online verfügbar: https://raw.githubusercontent.com/COST-ELTeC/Schemas/master/eltec-1.rng. Auch hier verweisen wir der Einfachheit halber auf eine lokale Kopie. 

Für das Validieren verwenden wir lxml; siehe die Dokumentation: https://lxml.de/. 

In [31]:
# === Importe ===

import os
import glob
from lxml import etree
import sys
from os.path import join
from os.path import basename as bn


# === Functions === 

def parse_xml(xmlfile): 
    with open(xmlfile, "r", encoding="utf8") as infile:
        parsed = etree.parse(infile)
        return parsed

def validate_xml(teiparsed, rngparsed, filename):
    # Validierung
    rngvalidator = etree.RelaxNG(rngparsed)
    validation = rngvalidator.validate(teiparsed)
    log = rngvalidator.error_log
    # Show results
    if validation == True: 
        print("OK, valid: " + filename)
    else:
        print("\nSORRY, not valid: " + filename + "\n" + str(log) + "\n")


# === Main ===

wdir = join("..", "data", "eltec-eng")
teifilepaths = join(wdir, "level1", "*.xml")
rngfilepath = join(wdir, "eltec-1.rng")


def main(teifilepaths, rngfilepath): 
    rngparsed = parse_xml(rngfilepath)
    for teifilepath in glob.glob(teifilepaths):
        filename = str(bn(teifilepath).split(".")[0])
        teiparsed = parse_xml(teifilepath)
        validate_xml(teiparsed, rngparsed, filename)
 
main(teifilepaths, rngfilepath)


OK, valid: ENG18470_Aguilar

SORRY, not valid: ENG18400_Trollope
/home/christof/Repositories/Github/dh-trier/textprozessieren/data/eltec-eng/level1/ENG18400_Trollope.xml:9:0:ERROR:RELAXNGV:RELAXNG_ERR_ELEMWRONG: Did not expect element author there

OK, valid: ENG18450_Disraeli
OK, valid: ENG18460_Reynolds
OK, valid: ENG18410_Sinclair
OK, valid: ENG18471_Bronte
OK, valid: ENG18440_Disraeli
OK, valid: ENG18411_Tupper


### Aus XML Metadaten oder plain text extrahieren: XPath in lxml

Für diesen Anwendungsfall verwenden wir die ELTeC-Sammlungen (genauer: Ebenfalls die 8 Beispieldateien aus ELTeC-eng) als Datensatz. Die folgenden Szenarien werden betrachtet: 

1. Die Extraktion von Metadaten aus dem TEI Header, um eine Metadatentabelle anzulegen
2. Die Transformation des Textes zu plain text, um weiterführende Textanalysen zu machen

### (1) Metadaten aus dem TEI Header extrahieren

In [32]:
# === Importe

import os
import re
import glob
from os.path import join
from os.path import basename
import pandas as pd
from lxml import etree
from collections import Counter


# === Functions ===


def open_file(xmlfile): 
    """
    Open and parse the XML file. 
    Returns an XML tree.
    """
    with open(xmlfile, "r", encoding="utf8") as infile:
        xml = etree.parse(infile)
        return xml


def get_metadatum(xml, xpath): 
    """
    For each XPath, get the metadata item from the XML tree.
    Returns a list.
    """
    try: 
        namespaces = {'tei':'http://www.tei-c.org/ns/1.0',
                      'eltec':'http://distantreading.net/eltec/ns'}       
        metadatum = xml.xpath(xpath, namespaces=namespaces)[0]
    except: 
        metadatum = "NA"
    metadatum = re.sub("\.", "", metadatum)
    return metadatum


def save_metadata(metadata): 
    """
    Save all metadata to a CSV file. 
    """
    metadatafile = "ELTeC-eng_metadata.tsv"
    metadata_df = pd.DataFrame(metadata)
    print(metadata_df)
    with open(metadatafile, "w", encoding="utf8") as outfile: 
        metadata_df.to_csv(outfile, sep="\t")

        
# === Main ===

teifilepaths = join("..", "data", "eltec-eng", "level1", "*.xml")

xpaths = {
    "xmlid" : "//tei:TEI/@xml:id", 
    "numwords" : "//tei:extent/tei:measure[@unit='words']/text()",
    "sizeCat" : "//tei:textDesc/eltec:size/@key",
    "firsted-yr" : "//tei:bibl[@type='firstEdition']/tei:date/text()",
    "time-slot" : "//tei:textDesc/eltec:timeSlot/@key"
    }


def main(teifilepaths, xpaths):
    allmetadata = []
    for teifilepath in glob.glob(teifilepaths): 
        filename,ext = basename(teifilepath).split(".")
        try: 
            keys = ["filename"]
            metadata = [filename]
            xml = open_file(teifilepath)
            for key,xpath in xpaths.items(): 
                metadatum = get_metadatum(xml, xpath)
                keys.append(key)
                metadata.append(metadatum)
            allmetadata.append(dict(zip(keys, metadata)))
        except: 
            print("ERROR!!!", filename)
    save_metadata(allmetadata)
    
main(teifilepaths, xpaths)


            filename     xmlid numwords sizeCat firsted-yr time-slot
0   ENG18470_Aguilar  ENG18470   171250    long       1847        T1
1  ENG18400_Trollope  ENG18400   207300    long         NA        T1
2  ENG18450_Disraeli  ENG18450   158160    long       1845        T1
3  ENG18460_Reynolds  ENG18460   839895    long       1846        T1
4  ENG18410_Sinclair  ENG18410   188876    long       1841        T1
5    ENG18471_Bronte  ENG18471   115398    long       1847        T1
6  ENG18440_Disraeli  ENG18440   159046    long       1844        T1
7    ENG18411_Tupper  ENG18411    34573   short       1844        T1


### Plain text aus einer XML-Datei extrahieren

Wir bleiben bei den Beispieldaten und wollen jetzt statt Metadaten aber den Textinhalt extrahieren. 

Dabei kann man, statt einfach nur alle Elemente zu löschen und den verbleibenden Text zu nehmen, natürlich im XML-Dokument filtern. So kann man beispielsweise festlegen, dass der `teiHeader` komplett ignoriert wird und im `text` beispielsweise nur der Inhalt von `body`, und zwar ohne die Überschriften (`head`) verwendet wird. 

Im vorliegenden, leicht vereinfachten Beispiel ist das nicht ausführlich parametrisierbar, das ist aber selbstverständlich umsetzbar. Im realen Anwendungsbeispiel kann auch eine automatische Modernisierung der Orthografie vorgenommen werden. Siehe hier: https://github.com/COST-ELTeC/Scripts/tree/master/Python (tei2txt). 

In [33]:
# === Importe

import os.path
import glob
from os.path import join
from lxml import etree
import pandas as pd
import re


# === Functions 

def read_teifile(teifile): 
    with open(teifile, "r", encoding="utf8") as outfile: 
        tei = etree.parse(teifile)
        return tei


def remove_tags(tei, nsp): 
    namespaces = nsp
    etree.strip_tags(tei, "tei:hi")
    etree.strip_tags(tei, "tei:foreign")
    etree.strip_tags(tei, "tei:quote")
    return tei


def remove_elements(tei, nsp): 
    namespaces = nsp
    etree.strip_elements(tei, "tei:head", with_tail=False)
    etree.strip_elements(tei, "tei:note", with_tail=False)
    return tei
    

def get_text(tei, nsp): 
    xpath = "//tei:body//text()"
    text = tei.xpath(xpath, namespaces=nsp)
    text = " ".join(text)
    return text


def clean_text(text): 
    text = re.sub("[ ]{2,20}", " ", text)
    text = re.sub("\n{2,20}", "\n", text)
    text = re.sub("[ \n]{2,20}", " \n", text)
    text = re.sub("\t{1,20}", "\t", text)
    return text
    

def save_text(text, txtpath, filename): 
    filename = join(txtpath, filename+".txt")
    with open(filename, "w", encoding="utf8") as outfile: 
        outfile.write(text)


# Parameters 

teipaths = join("..", "data", "eltec-eng", "level1", "*.xml")
txtpath = join("..", "data", "eltec-eng", "plaintext", "")
nsp = {'tei':'http://www.tei-c.org/ns/1.0'}
        

# === Main 

def main(teipaths, txtpath, nsp): 
    if not os.path.exists(txtpath):
        os.makedirs(txtpath)
    for teifile in glob.glob(teipaths):
        filename,ext = os.path.basename(teifile).split(".")
        tei = read_teifile(teifile)
        tei = remove_tags(tei, nsp)
        tei = remove_elements(tei, nsp)
        text = get_text(tei, nsp)
        text = clean_text(text)
        print(text[0:500])
        save_text(text, txtpath, filename)

main(teipaths, txtpath, nsp)


 
PART I. 
THE SISTERS. 
CHAPTER I. 
A LAUNCH.—A PROMISE.—A NEW RELATION. 
In a very beautiful part of Wales, between the northern boundaries of Glamorgan and the 
southeastern of Carmarthenshire, there stood, some twenty or thirty years ago, a small 
straggling village. Its locality was so completely concealed that the appearance of a 
gentleman's carriage, or, in fact, any vehicle superior to a light spring-cart, was of such 
extremely rare occurence as to be dated, in the annals of Llangwilla
 
CHAPTER I. 
DESCRIPTION OF DOWLING LODGE AND ITS APPURTENANCES — OF ITS MASTER — OF ITS MISTRESS 
— AND ALL THE MASTERS AND MISSES DOWLING — A LARGE DINNER-PARTY — A HOT 
DRAWING-ROOM, AND THE WAY TO ESCAPE FROM IT. 
No traveller can ride or drive within sight of Dowling Lodge, without being tempted 
to inquire, "Whose house is that?" 
It forms, indeed, a very striking object on the right of the London road, as the hill 
rises gradually, and overlooks the town of Ashleigh, one of the busiest 

## XSL-Transformation auf XML anwenden mit lxml

Das Beispiel kommt aus einer der vergangenen Sitzungen, in denen Sie mit einem Editor eine XSL-Transformation vorgenommen haben (Baudelaire-Beispiel). Das geht auch mit lxml, wobei lxml aber nur XSLT 1.0 unterstützt, also relativ einfache Stylesheets. 

In [34]:
from os.path import join
import lxml.etree as et

sourcefile = join("..", "data", "xslt", "source.xml")
xsltfile = join("..", "data", "xslt", "transform.xsl")
outputfile = "output.html"

def transform_xml(sourcefile, xsltfile, outputfile): 

    # Read and parse XML sourcefile and XSLT stylsheet
    with open(sourcefile, "r", encoding="utf8") as infile:
        xml = et.parse(infile)
    with open(xsltfile, "r", encoding="utf8") as infile:
        xslt = et.parse(infile)

    # Instantiate the transformer and apply to XML
    transformer = et.XSLT(xslt)
    result = transformer(xml)
    print(result)
    
    # Transform to string and save the result to disk
    with open(outputfile, "w", encoding="utf8") as outfile:
        outfile.write(str(result))
    
transform_xml(sourcefile, xsltfile, outputfile)

<html>
<head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<meta name="Erscheinungsjahr" content="">
</head>
<body>
<h2>
          Charles
          Baudelaire
        </h2>
<h4>À une passante</h4>
<p>La rue assourdissante autour de moi hurlait.<br>Longue, mince, en grand deuil, douleur majestueuse,<br>Une femme passa, d’une main fastueuse<br>Soulevant, balançant le feston et l’ourlet ;<br></p>
<p>Agile et noble, avec sa jambe de statue.<br>Moi, je buvais, crispé comme un extravagant,<br>Dans son oeil, ciel livide où germe l’ouragan,<br>La douceur qui fascine et le plaisir qui tue.<br></p>
<p>Un éclair… puis la nuit ! – Fugitive beauté<br>Dont le regard m’a fait soudainement renaître,<br>Ne te verrai-je plus que dans l’éternité ?<br></p>
<p>Ailleurs, bien loin d’ici ! trop tard ! jamais peut-être !<br>Car j’ignore où tu fuis, tu ne sais où je vais,<br>Ô toi que j’eusse aimé, ô toi qui le savais !<br></p>
</body>
</html>



## Dokumentation generieren: von RNG nach Markdown

Dieser schon etwas speziellere Anwendungsfall betrifft das automatische Generieren einer menschenlesbaren Dokumentation aus einer Schema-Datei, die in RNG (XML-Syntax) geschrieben ist. 

Hier ist der reale Code etwas komplexer, als es sich für eine detaillierte Erklärung anbietet. Daher hier nur einige Screenshots sowie einige Links, unter denen Sie sich das anschauen können: 

* RelaxNG-Datei (XML-Syntax): https://github.com/dh-trier/wlv/blob/master/schemas/wlv-label-schema.rng
* Python-Skript zur Transformation: https://github.com/dh-trier/wlv/blob/master/schemas/make-schema-docs.py
* "Einleitung" in Markdown: https://raw.githubusercontent.com/dh-trier/wlv/master/schemas/wlv-introduction.md
* Dokumentation im Ergebnis (nach unten scrollen, um den generierten TEil zu sehen): https://github.com/dh-trier/wlv/blob/master/schemas/wlv-label-docs.md

Es ist eigentlich erstaunlich, dass es für diesen Use Case keine Standard-Library gibt. Falls Sie hier etwas entdecken, freue ich mich über Hinweise.  

#### Ausschnitt aus dem Schema in RelaxNG

![RNG2MD](rng2md-1.png)

#### Ausschnitt aus dem Python-Skript zur Umwandlung

![RNG2MD](rng2md-2.png)

#### Ausschnitt aus der fertigen Dokumentation (Markdown links, Preview rechts)

![RNG2MD](rng2md-3.png)

## Fazit

### Markup und Python

1. Arbeiten besser zusammen, als man vielleicht manchmal denkt
1. Es gibt eine Reihe von Libraries, die man dafür kennen sollte
1. Immer wenn man mehr als eine Handvoll Dateien bearbeiten möchte, oder wenn man eine Routine mehrfach anwenden möchte, lohnt sich der Aufwand der Automatisierung mit Python